In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json

In [2]:
# constants
DATA_PATH = '../data/'
TRANSPORT_DATA_PATH = DATA_PATH + 'transport/'
GENERATED_DATA_PATH = '../generated/'

In [3]:
sea = pd.read_csv(TRANSPORT_DATA_PATH + 'Belgium_Import_Sea_Data.csv')
air = pd.read_csv(TRANSPORT_DATA_PATH + 'Belgium_Import_Air_Data.csv')
road = pd.read_csv(TRANSPORT_DATA_PATH + 'Belgium_Import_Road_Data.csv')

In [4]:
# Append the three dataframes
transport_merge = sea.append(air.append(road))

In [5]:
sea

,PERIOD,REPORTER,PARTNER,PRODUCT,FLOW,TRANSPORT_MODE,INDICATORS,Value,Flag and Footnotes
0,Jan.-Dec. 2000,BELGIUM (and LUXBG -> 1998),CHINA (PEOPLE'S REPUBLIC OF),TOTAL,1,Sea,QUANTITY_IN_TONS,"1,377,800",NaN
1,Jan.-Dec. 2000,BELGIUM (and LUXBG -> 1998),CHINA (PEOPLE'S REPUBLIC OF),AGRICULTURAL PRODUCTS AND LIVE ANIMALS,1,Sea,QUANTITY_IN_TONS,"8,876",NaN
2,Jan.-Dec. 2000,BELGIUM (and LUXBG -> 1998),CHINA (PEOPLE'S REPUBLIC OF),FOODSTUFFS AND ANIMAL FODDER,1,Sea,QUANTITY_IN_TONS,"27,795",NaN
3,Jan.-Dec. 2000,BELGIUM (and LUXBG -> 1998),UNITED STATES,TOTAL,1,Sea,QUANTITY_IN_TONS,"6,292,943",NaN
4,Jan.-Dec. 2000,BELGIUM (and LUXBG -> 1998),UNITED STATES,AGRICULTURAL PRODUCTS AND LIVE ANIMALS,1,Sea,QUANTITY_IN_TONS,"292,460",NaN
...,...,...,...,...,...,...,...,...,...
15595,Jan.-Dec. 2019,BELGIUM (and LUXBG -> 1998),ZIMBABWE (RHODESIA ->1980),AGRICULTURAL PRODUCTS AND LIVE ANIMALS,1,Sea,QUANTITY_IN_TONS,:,NaN
15596,Jan.-Dec. 2019,BELGIUM (and LUXBG -> 1998),ZIMBABWE (RHODESIA ->1980),FOODSTUFFS AND ANIMAL FODDER,1,Sea,QUANTITY_IN_TONS,:,NaN
15597,Jan.-Dec. 2019,BELGIUM (and LUXBG -> 1998),"NO DATA, WORK CODE",TOTAL,1,Sea,QUANTITY_IN_TONS,:,NaN
15598,Jan.-Dec. 2019,BELGIUM (and LUXBG -> 1998),"NO DATA, WORK CODE",AGRICULTURAL PRODUCTS AND LIVE ANIMALS,1,Sea,QUANTITY_IN_TONS,:,NaN


In [6]:
def clean_data(df):
    df = df.drop(columns=['FLOW', 'Flag and Footnotes', 'INDICATORS', 'REPORTER'])
    # Make year pretty
    df['PERIOD'] = df['PERIOD'].apply(lambda x: x.split(' ')[1])
    
    df = df[df['PRODUCT'] == "TOTAL"]
    df = df[df['Value'] != ":"]
    df = df.drop(columns=['PRODUCT'])
    return df

In [7]:
transport = clean_data(transport_merge)

In [8]:
transport['PARTNER'].unique()

array(["CHINA (PEOPLE'S REPUBLIC OF)", 'UNITED STATES', 'ANDORRA',
       'UNITED ARAB EMIRATES', 'ANGUILLA', 'ALBANIA', 'ARMENIA',
       'NETHERLANDS ANTILLES (incl. ARUBA->1986)', 'ANGOLA', 'ARGENTINA',
       'AUSTRALIA', 'ARUBA', 'AZERBAIJAN', 'BOSNIA AND HERZEGOVINA',
       'BANGLADESH', 'BURKINA FASO  (UPPER VOLTA -> 1985)', 'BULGARIA',
       'BAHRAIN', 'BURUNDI', 'BENIN (DAHOMEY -> 1976)', 'BOLIVIA',
       'BRAZIL', 'BAHAMAS', 'BOTSWANA', 'BELARUS (BELORUSSIA)', 'BELIZE',
       'CANADA', 'CONGO, DEMOCRATIC REPUBLIC OF (ZAIRE ->1997)',
       'CENTRAL AFRICAN REPUBLIC', 'CONGO',
       'SWITZERLAND (incl. LI->1994)', "COTE D'IVOIRE", 'CHILE',
       'CAMEROON', 'COLOMBIA', 'COSTA RICA', 'CUBA', 'CAPE VERDE',
       'CYPRUS', 'CZECHIA (CS->1992)', 'DJIBOUTI (AFARS ISSAS->1977)',
       'DOMINICAN REPUBLIC', 'ALGERIA', 'ECUADOR', 'ESTONIA', 'EGYPT',
       'ERITREA', 'ETHIOPIA (incl. ERITREA ->1993)', 'FIJI',
       'FALKLAND ISLANDS', 'FAROE ISLANDS', 'GABON', 'GRENADA', 'GEO

We have to clean up the country names so we can calculate the distances using the GeoDist DataSet 

In [9]:
def clean_strings(series):
    # Clean everything between brackets and remove trailing whitespaces
    new_series = series.apply(lambda x: x.split('(')[0].strip())
    # Remove everything after comma
    new_series = new_series.apply(lambda x: x.split(',')[0].strip().lower())
    return new_series

In [10]:
transport_clean = transport
transport_clean['PARTNER'] = clean_strings(transport_clean['PARTNER'])

In [11]:
# Save to CSV to clean
transport.to_csv(TRANSPORT_DATA_PATH + 'transport_clean.csv')

Now we calculate the distance between Belgium and every country

In [12]:
countries = pd.read_excel(TRANSPORT_DATA_PATH+'geo_cepii.xls') 
countries

,iso2,iso3,cnum,country,pays,area,dis_int,landlocked,continent,city_en,...,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
0,AW,ABW,533,Aruba,Aruba,193,5.225315,0,America,Oranjestad,...,.,.,.,NLD,.,.,.,.,.,.
1,AF,AFG,4,Afghanistan,Afghanistan,652225,303.761400,1,Asia,Kabul,...,Uzbek,.,.,.,.,.,.,GBR,.,.
2,AO,AGO,24,Angola,Angola,1246700,419.966600,0,Africa,Luanda,...,.,.,.,PRT,.,.,.,.,.,.
3,AI,AIA,660,Anguilla,Anguilla,102,3.798690,0,America,The Valley,...,.,.,.,GBR,.,.,.,.,.,.
4,AL,ALB,8,Albania,Albanie,28748,63.773110,0,Europe,Tirana,...,.,.,.,TUR,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,ZA,ZAF,711,South Africa,Afrique du Sud,1219912,415.430200,0,Africa,Pretoria,...,Shoto,.,.,GBR,NLD,.,.,.,.,.
234,ZA,ZAF,711,South Africa,Afrique du Sud,1219912,415.430200,0,Africa,Cape Town,...,Shoto,.,.,GBR,NLD,.,.,.,.,.
235,ZR,ZAR,180,Congo (Democratic Republic of the),"Congo, Rép. dém. du",2345410,576.027500,0,Africa,Kinshasa,...,Luba-Kasai,French,.,BEL,.,.,.,.,.,.
236,ZM,ZMB,894,Zambia,Zambie,752614,326.302200,1,Africa,Lusaka,...,Tonga,.,.,GBR,.,.,.,.,.,.


In [13]:
distances = pd.read_excel(TRANSPORT_DATA_PATH+'dist_cepii.xls') 
distances

,iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces
0,ABW,ABW,0,0,0,0,0,0,0,0,5.225315,5.225315,25.0935,23.0472
1,ABW,AFG,0,0,0,0,0,0,0,0,13257.810000,13257.810000,13168.2,13166.4
2,ABW,AGO,0,0,0,0,0,0,0,0,9516.913000,9516.913000,9587.32,9584.19
3,ABW,AIA,0,0,1,0,0,0,0,0,983.268200,983.268200,976.897,976.892
4,ABW,ALB,0,0,0,0,0,0,0,0,9091.742000,9091.742000,9091.58,9091.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50171,ZWE,YUG,0,0,0,0,0,0,0,0,7056.402000,7056.402000,7046.43,7042.83
50172,ZWE,ZAF,1,1,1,0,0,0,0,0,2186.206000,926.174300,1258.55,1101.44
50173,ZWE,ZAR,0,0,0,0,0,0,0,0,2283.061000,2283.061000,1930.98,1719.15
50174,ZWE,ZMB,1,1,1,0,1,0,0,0,396.804100,396.804100,583.795,525.073


In [14]:
countries = countries[['iso3','country']]

In [15]:
distances = distances[distances['iso_o'] == 'BEL']

In [16]:
# Take distance from Belgium to capital of the country
distances = distances[['iso_d','distcap']]

In [17]:
belgium_distances = distances.merge(countries,left_on='iso_d',right_on='iso3')

In [18]:
belgium_distances = belgium_distances[['country', 'distcap']]
belgium_distances['country'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Andorra',
       'Netherland Antilles', 'United Arab Emirates', 'Argentina',
       'Armenia', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium and Luxembourg', 'Benin',
       'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan',
       'Botswana', 'Central African Republic', 'Canada',
       'Cocos (Keeling) Islands', 'Switzerland', 'Chile', 'China',
       "Côte d'Ivoire", 'Cameroon', 'Congo', 'Cook Islands', 'Colombia',
       'Comoros', 'Cape Verde', 'Costa Rica', 'Cuba', 'Christmas Island',
       'Cayman Islands', 'Cyprus', 'Czech Republic', 'Germany',
       'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic', 'Algeria',
       'Ecuador', 'Egypt', 'Eritrea', 'Western Sahara', 'Spain',
       'Estonia', 'Ethiopia', 'Finland', 'Fi

In [19]:
belgium_distances

,country,distcap
0,Aruba,7847.070
1,Afghanistan,5416.121
2,Angola,6696.229
3,Anguilla,6876.788
4,Albania,1589.107
...,...,...
232,South Africa,8838.753
233,South Africa,8838.753
234,Congo (Democratic Republic of the),6224.989
235,Zambia,7738.871


In [20]:
transport_clean


,PERIOD,PARTNER,TRANSPORT_MODE,Value
0,2000,china,Sea,"1,377,800"
3,2000,united states,Sea,"6,292,943"
6,2000,andorra,Sea,2
9,2000,united arab emirates,Sea,"400,621"
18,2000,anguilla,Sea,4
...,...,...,...,...
14790,2018,serbia,Road,"59,706"
14799,2018,yemen,Road,194
14808,2018,south africa,Road,"72,021"
14811,2018,zambia,Road,883


In [21]:
belgium_distances_clean = belgium_distances
belgium_distances_clean['country'] = clean_strings(belgium_distances['country'])

Now, we take the set of the countries, and look which do and do not overlap with our transport dataset

In [22]:
dist_countries = set(belgium_distances_clean['country'])
trans_countries = set(transport_clean['PARTNER'])

In [23]:
# Rename all entries in dist_countries that we need manually
belgium_distances_clean.to_csv(TRANSPORT_DATA_PATH+'belgium_distances.csv', encoding='utf-16')

In [24]:
belgium_distances_clean = pd.read_csv(TRANSPORT_DATA_PATH+'belgium_distances_clean.csv').drop(columns='Column1')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [26]:
dist_countries = set(belgium_distances_clean['country'])
trans_countries = set(transport_clean['PARTNER'])

In [28]:
transport_clean

,PERIOD,PARTNER,TRANSPORT_MODE,Value
0,2000,china,Sea,"1,377,800"
3,2000,united states,Sea,"6,292,943"
6,2000,andorra,Sea,2
9,2000,united arab emirates,Sea,"400,621"
18,2000,anguilla,Sea,4
...,...,...,...,...
14790,2018,serbia,Road,"59,706"
14799,2018,yemen,Road,194
14808,2018,south africa,Road,"72,021"
14811,2018,zambia,Road,883


In [29]:
belgium_distances_clean

,country,distcap
0,aruba,7847.070
1,afghanistan,5416.121
2,angola,6696.229
3,anguilla,6876.788
4,albania,1589.107
...,...,...
232,south africa,8838.753
233,south africa,8838.753
234,congo,6224.989
235,zambia,7738.871


In [30]:
# Merge transport data with distances
transport_total = transport_clean.merge(belgium_distances_clean, left_on='PARTNER', right_on='country', how='left')
transport_total = transport_total.drop(columns='country')
transport_total['distcap'] = transport_total['distcap'].fillna(0)
transport_total['distcap'] = transport_total['distcap'].apply(lambda x: int(x))

In [37]:
transport_total['Value'] = transport_total['Value'].apply(lambda x: int(x.replace(',', '')))

Now we can use the data from https://www.sciencedirect.com/science/article/pii/S0959652615000438 (emission/(km*ton) to calculate the total emission

In [39]:
emissions = {'Sea':14,'Rail':18,'Road':75,'Air':690}

In [40]:
transport_total['Emission'] = 0

In [103]:
transport_total['Emission'] = transport_total.apply(lambda x: x['Value']*x['distcap']*emissions[x['TRANSPORT_MOD']], axis=1)

In [104]:
transport_total

,PERIOD,PARTNER,TRANSPORT_MODE,Value,distcap,Emission
0,2000,china,Sea,"1,377,800",7970,"1,377,8001,377,8001,377,8001,377,8001,377,8001..."
1,2000,united states,Sea,"6,292,943",6222,"6,292,9436,292,9436,292,9436,292,9436,292,9436..."
2,2000,andorra,Sea,2,952,2222222222222222222222222222222222222222222222...
3,2000,united arab emirates,Sea,"400,621",5159,"400,621400,621400,621400,621400,621400,621400,..."
4,2000,anguilla,Sea,4,6876,4444444444444444444444444444444444444444444444...
...,...,...,...,...,...,...
10446,2018,yemen,Road,194,5307,1941941941941941941941941941941941941941941941...
10447,2018,south africa,Road,"72,021",8838,"72,02172,02172,02172,02172,02172,02172,02172,0..."
10448,2018,south africa,Road,"72,021",8838,"72,02172,02172,02172,02172,02172,02172,02172,0..."
10449,2018,zambia,Road,883,7738,8838838838838838838838838838838838838838838838...
